In [5]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import xarray as xr

In [6]:
dirname = '/Users/gabriel/Documents/princeton/courses/spring_2025/cos_513/assignments/project/data'
search_substring = 'net_energy_load'
file_extension = 'processed.xls'

filenames = [os.path.join(dirname, f) for f in os.listdir(dirname) if
             f.endswith(file_extension) and
             search_substring in f]

In [7]:
container = []
column_names = ['estimate_type', 'estimate_generation_year', 'year', 
                'ALL_US', 'FRCC', 'MRO', 'NPCC', 'RFC', 'SERC', 'SPP', 'ERCOT', 'WECC']
regions = ['ALL_US', 'FRCC', 'MRO', 'NPCC', 'RFC', 'SERC', 'SPP', 'ERCOT', 'WECC']
for filename in filenames:
    temp = pd.read_excel(filename, names=column_names).dropna()
    container.append(temp)

dataset = pd.concat(container)

In [8]:
power_generation_actual = dataset.loc[dataset['estimate_type'] == 'Actual'].sort_values('estimate_generation_year')
power_generation_actual = power_generation_actual.drop(columns=['estimate_type', 'estimate_generation_year']).set_index('year')
power_generation_actual = power_generation_actual.astype({c: 'float' for c in power_generation_actual.columns})

container = []
for nerc_region in power_generation_actual.columns:
    DF_TMP = pd.DataFrame(power_generation_actual[nerc_region]).reset_index(names=['year'])
    DF_TMP = DF_TMP.rename(columns={nerc_region: 'LOAD'})
    DF_TMP['region'] = nerc_region
    container.append(DF_TMP)
TMP = pd.concat(container)

dataset_name = 'EIA'
field_name = 'net_energy_load'
timestamp_to_str = lambda x: f'{int(x)}0101'
start_date_str = timestamp_to_str(power_generation_actual.index.min())
end_date_str = timestamp_to_str(power_generation_actual.index.max())

storage_dirname = dirname
storage_filename = f'dataset-{dataset_name}.field_name-{field_name}.start_date-{start_date_str}.end_date-{end_date_str}.nc'
storage_pathname = os.path.join(storage_dirname, storage_filename)
TMP.set_index(['region', 'year']).to_xarray().to_netcdf(storage_pathname)